## Project - Airline AI Assistant 

In [1]:
from openai import OpenAI 
import gradio as gr 
import json 

In [2]:
!ollama list

NAME                       ID              SIZE      MODIFIED     
qwen2.5-coder:latest       dae161e27b0e    4.7 GB    2 weeks ago     
gpt-oss:latest             17052f91a42e    13 GB     2 weeks ago     
llama3.2:3b                a80c4f17acd5    2.0 GB    3 weeks ago     
llama3.1:8b                46e0c10c039e    4.9 GB    3 months ago    
nomic-embed-text:latest    0a109f422b47    274 MB    5 months ago    


In [ ]:
MODEL = "gpt-oss:latest"

OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

In [31]:
system_message = """
You are a helpful assistant for an airline called FlightAI.

Rules:
- For greetings, small talk, or general questions, respond normally in plain text.
- ONLY call a tool when the user explicitly asks for ticket prices or costs.
- NEVER mention tools, functions, or internal reasoning to the user.
- NEVER fabricate tool calls.
- If no tool is required, answer directly.

Responses must be funny, short, and no more than 1 sentence.
"""


In [6]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination):
    print("tool called for city {destination}")
    price = ticket_prices.get(destination.lower(), "Sorry, we don't have tickets to that destination.")
    return f"The price for a ticket to {destination} is {price}."


In [7]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [9]:
tools = [{"type":"function", "function": price_function}]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

## Getting OpenAI to used out internal function as tool 

In [35]:
def chat(messsage, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messsages = [{"role":"system", "content":system_message}] + history + [{"role":"user", "content":messsage}]
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=messsages,
        tools=tools
    )

    print(response.choices[0].message)

    while response.choices[0].message.tool_calls:
        messsage = response.choices[0].message
        response = handle_tool_call(messsage)
        messsages.append(messsage)
        messsages.extend(response)
        response = ollama.chat.completions.create(
            model=MODEL,
            messages=messsages,
            tools=tools
        )
        
    
    return response.choices[0].message.content
    


In [10]:
def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            args = json.loads(tool_call.function.arguments)
            destination_city = args.get("destination_city")
            price_details = get_ticket_price(destination_city)
            responses.append({
                "role":"tool",
                "content":price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [ ]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


ChatCompletionMessage(content='Hello, how can I assist you with your flight to paradise?', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)
ChatCompletionMessage(content="No function call required for this question, we're just small-talking!\n\nHowever if you'd like ticket prices:\n\nWhat's your destination city?", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_nrqoeo7d', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function', index=0), ChatCompletionMessageFunctionToolCall(id='call_8jqrc8ps', function=Function(arguments='{"destination_city":"Paris"}', name='get_ticket_price'), type='function', index=1), ChatCompletionMessageFunctionToolCall(id='call_dydcyey9', funct

## sqLite for database

In [15]:
import sqlite3

In [16]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (destination TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [17]:
def get_ticket_price(destination):
    print(f"DATABASE TOOL CALLED: Getting price for {destination}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE destination = ?', (destination.lower(),))
        result = cursor.fetchone()
        return f"ticket price to {destination} is ${result[0]}" if result else f"Sorry, we don't have tickets to {destination}."

In [18]:
get_ticket_price("London")

DATABASE TOOL CALLED: Getting price for London


'ticket price to London is $799.0'

In [19]:
def set_ticket_price(destination, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (destination, price) VALUES (?, ?) ON CONFLICT(destination) DO UPDATE SET price = ?', (destination.lower(), price, price))
        conn.commit()

In [20]:
ticket_prices = {"london":799, "paris": 899, "tokyo": 1420, "sydney": 2999}
for destination, price in ticket_prices.items():
    set_ticket_price(destination, price)

In [21]:
get_ticket_price("London")

DATABASE TOOL CALLED: Getting price for London


'ticket price to London is $799.0'

In [ ]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


ChatCompletionMessage(content='Hello! How can I help you today?', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning='User says "hey". It\'s a greeting. We respond with a greeting. No tool needed. Short, courteous, <=1 sentence.')
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_xqe9l8bw', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function', index=0)], reasoning='The user explicitly asks ticket price to London. Must call the get_ticket_price function with destination_city "London".')
DATABASE TOOL CALLED: Getting price for London
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_vbg8tiyh', function=Function(arguments='{"destination_